## **ALGORITMA APRIORI UNTUK PREDIKSI HUBUNGAN ANTAR PEMBELIAN BARANG DI TOKO KELONTONG**

# Mengimport Packages

In [97]:
import pandas as pd
from matplotlib import pyplot as plt
!pip install apyori
from apyori import apriori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Mengimport dataset menggunakan dataframe pandas

In [98]:
df = pd.read_csv('Groceries_dataset.csv')

# Mereview Dataset 

In [99]:
df.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


# Memeriksa Apakah Dataframe Ada Nilai Null

In [100]:
df.isnull().sum()

Member_number      0
Date               0
itemDescription    0
dtype: int64

# Mengubah Nama Kolom Pada Dataset

In [101]:
df.columns = ['member_id','date','items']

#Mengubah Tipe Data Pada Dataframe Dan Menampilkan Informasi Detail Tentang Data Frame

In [102]:
df.info()
df['date']= pd.to_datetime(df['date'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   member_id  38765 non-null  int64 
 1   date       38765 non-null  object
 2   items      38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB


# Menentukan Item Yang Paling Banyak Dibeli

In [103]:
item_count=df[['member_id','items']].groupby('items',as_index=False)['member_id'].count(
    ).sort_values(by='member_id',ascending=False)
item_count.rename(columns = {'member_id':'count'},inplace=True)
item_count.head(3)

,items,count
164,whole milk,2502
102,other vegetables,1898
122,rolls/buns,1716


# Menentukan Rata-Rata Jumlah Item Yang Dibeli Setiap Member

In [104]:
member_items = df.groupby('member_id', as_index=False)['items'].count()
print('On average, members purchased: {x} items.'.format(
    x= round((member_items['items'].mean()),1)))

On average, members purchased: 9.9 items.


#Membuat Dataframe Baru Yang Berisikan List Pada Item Yang Dibeli Pada Setiap Member Setiap Harinya

In [105]:
basket_df = df.groupby(['member_id', 'date'], as_index=True)['items'].apply(lambda x: list(x))

transactions = basket_df.values.tolist()

transactions[:10]

[['whole milk', 'pastry', 'salty snack'],
 ['sausage', 'whole milk', 'semi-finished bread', 'yogurt'],
 ['soda', 'pickled vegetables'],
 ['canned beer', 'misc. beverages'],
 ['sausage', 'hygiene articles'],
 ['sausage', 'whole milk', 'rolls/buns'],
 ['whole milk', 'soda'],
 ['frankfurter', 'soda', 'whipped/sour cream'],
 ['frankfurter', 'curd'],
 ['beef', 'white bread']]

#Membuat Aturan Asosiasi Yang Dimana Aturan Ini Berfungsi Dapat Menghasilkan Aturan Asosiasi Yang Memenuhi Kualifikasi.

untuk nilai minimal yang ditetapkan semakin tinggi maka semakin sedikit aturan asosiasi yang memenuhi

In [106]:
association_rules = apriori(transactions,
                            min_support=0.0005,
                            min_lift=2,
                            min_confidence=0.04,
                            min_length=2)
association_results = list(association_rules)
print(len(association_results))

12


#Membuat Row Dan Columns Yang Akan Digunakan

In [107]:
results = []
for item in association_results:
    itemset = item[0]
    items = [x for x in itemset]

    value0 = str(items[0])
    value1 = str(items[1])
    value3 = str(item[1])[:7]
    value4 = str(item[2][0][2])[:7]
    value5 = str(item[2][0][3])[:7]

    rows = (value0, value1, value3, value4, value5)
    results.append(rows)
    columns = ['item_x', 'item_y', 'support', 'confidence', 'lift']

    associations_df = pd.DataFrame(results, columns=columns)
    associations_df.head(5)

#Menentukan Nilai Support Tetinggi Beserta Lift

In [108]:
associations_df=associations_df.sort_values(by=['support', 'lift'],
                            ascending=[False, False]).head(100)
associations_df.head(20)

,item_x,item_y,support,confidence,lift
11,yogurt,whole milk,0.00147,0.13173,2.18291
0,whole milk,brandy,0.00086,0.34210,2.16628
6,sweet spreads,tropical fruit,0.00073,0.16176,2.38706
3,soft cheese,hamburger meat,0.00060,0.06000,2.74550
9,whole milk,sausage,0.00060,0.39130,2.47781
10,rolls/buns,yogurt,0.00060,0.20454,2.38180
5,spices,soda,0.00060,0.225,2.31705
2,pickled vegetables,ham,0.00053,0.05970,3.48950
4,sweet spreads,pip fruit,0.00053,0.11764,2.39830
1,frankfurter,mustard,0.00053,0.08695,2.30288


#Menentukan Nilai Max Support Yang Dimana Dapat Digunakan Untuk Mengetahui Item Yang Sering Dibeli Bersamaan 

In [109]:
associations_df.sort_values(by='support',ascending=False).head(1)

,item_x,item_y,support,confidence,lift
11,yogurt,whole milk,0.00147,0.13173,2.18291


#Menentukan Nilai Max Lift Yang Dimana Dapat Digunakan Untuk Mengetahui Item Yang Sering Dibeli Bersamaan 

In [110]:
associations_df.sort_values(by='lift',ascending=False).head(1)

,item_x,item_y,support,confidence,lift
2,pickled vegetables,ham,0.00053,0.05970,3.48950
